# 4. Код для сервера (для отладки)

In [1]:
import pandas as pd
import pickle
import numpy as np
from statistics import mean
import re 

Проверять работу прототипа будем на данных из имеющегося датасета

In [2]:
#загружаем данные 
data_source = pd.read_csv('data.csv')

In [3]:
#На вход у нас строка с данными
stroka = data_source[data_source['target'].isna()].iloc[0]

Подгружаем все нужные нам файлы и модель

In [4]:
with open('mean_target.txt', 'rb') as f:
    mean_target = pickle.load(f)

In [5]:
mean_target

state
AL    7.200000e+04
AZ    3.306392e+05
BA    2.250000e+05
CA    1.403847e+06
CO    5.858594e+05
DC    9.335807e+05
DE    4.818378e+05
FL    5.702715e+05
Fl    1.740000e+05
GA    4.992980e+05
IA    1.738213e+05
IL    4.619961e+05
IN    2.729528e+05
KY    1.739727e+05
MA    1.454522e+06
MD    6.433096e+05
ME    1.549625e+05
MI    1.416609e+05
MO    2.496001e+05
MS    2.208450e+05
MT    2.209714e+05
NC    4.034450e+05
NJ    3.763549e+05
NV    5.266701e+05
NY    1.839118e+06
OH    2.328357e+05
OK    1.235571e+05
OR    5.866624e+05
OS    1.250000e+05
OT    5.000000e+04
PA    3.728949e+05
SC    3.606109e+05
TN    4.158094e+05
TX    4.617258e+05
UT    5.857265e+05
VA    5.433144e+05
VT    3.565737e+05
WA    7.150374e+05
WI    2.666345e+05
Name: target, dtype: float64

In [6]:
with open('states_list.txt', 'rb') as f:
    states_list = pickle.load(f)

In [7]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

## Вспомогательные функции. Такие же, как использовались в ноутбуке 1. Features
!!!! Если будут какие-то изменения в фукнциях в 1 ноутбуке, то и в серверную часть придется вносить изменения

In [8]:
def replace_property(row):
    
    if row in ['single-family home','Single Family Home','Single Family']:
        return 'single family home'
    elif row == 'Condo':
        return 'condo'
    elif row in ['Townhouse', 'Townhome']:
        return 'townhouse'
    elif row in ['Multi-Family Home','multi-family','Multi Family','Multi-Family']:
        return 'multi family home'
    elif row in ['1 Story','One Story']:
        return '1 story'
    elif row in ['2 Stories', '2 Story', 'Two Story']:
        return '2 stories'
    elif row in ['mobile/manufactured', 'Mfd/Mobile Home', 'Mobile / Manufactured']:
        return 'mobile/manufactured'
    elif row in ['Cooperative', 'coop']:
        return 'cooperative'
    elif row in ['Apartment', 'apartment']:
        return 'apartment'
    elif row in ['lot/land', 'Land']:
        return 'land'
    elif row in ['Other Style', 'Other']:
        return 'other style'
    else:
        return row

In [9]:
def make_baths_count(row):
    row = str(row)
    row = row.replace('Bathrooms: ', '').replace(' Baths', '').replace(' ba', '').replace(',','.')
    try:
        if row != 'nan':
            return float(row)
        else:
            return 1
    except:
        return 1

In [10]:
def make_beds_count(row):
    row = str(row)
    if row.find(' Beds') > -1:
        try:
            return int(row.replace(' Beds', ''))
        except: #если указано что-то другое #1-2
            return 1
    elif row.find(' bd') >-1:
        try:
            return int(row.replace(' bd', ''))
        except: #если указано что-то другое #--
            return 0
    else: 
        try:
            return int(row)
        except:
            return 0

def make_beds_square(row): #переводим в квадратные метры
    row = str(row)
    if row.find(' sqft') > -1:
        try:
            return float(row.replace(' sqft', '').replace(',',''))*0.093
        except: 
            return 0
    elif row.find(' acres') > -1:
        try:
            return float(row.replace(' acres', '').replace(',',''))*4046.86
        except: 
            return 0
    else:
        return 0

In [11]:
def make_schools(row):

    #разбираем, какие классы есть в близлежащих школах
    grades = eval(row[1:-1])['data']['Grades']
    
    result={'K':0,'1':0,'2':0,'3':0,'4':0,'5':0,'6':0,'7':0,'8':0,'9':0,'10':0,'11':0,'12':0, 'PK':0}
    for i in grades:
        i = (str(i).replace(' to ', '-').replace('–', '-')).split(',')
        for ii in i:
            ii = ii.strip().upper().replace('PK','PRESCHOOL')
            if 'PRESCHOOL' in ii:
                result['PK']+=1
                ii = ii.replace('PRESCHOOL','1')
            if 'K' in ii:
                result['K']+=1
                ii = ii.replace('K','1')
            if '-' in ii:
                j = ii.split('-')
                for k in range(int(j[0]), int(j[1])+1):
                    result[str(k)]+=1
    
    #считаем количество школ 
    result['schools_count']=len(eval(row[1:-1])['rating'])
    
    #считаем дистанцию по школам
    result['min_dist']=0
    result['max_dist']=0
    result['mean_dist']=0

    data_list = eval(row[1:-1])['data']['Distance']
    dist_list=[]
    for i in data_list:
        i = i.replace('mi', '').strip()
        try:
            i = float(i)
            dist_list.append(i)
        except:
            print(i)
            i = 0 
    if len(dist_list)>0:
        result['min_dist']=min(dist_list)
        result['max_dist']=max(dist_list)
        result['mean_dist']=mean(dist_list)
    
    #разбираем рейтинги школ
    regex = re.compile('\D+')
    data2_list = eval(row[1:-1])['rating']
    rate_list=[]
    if len(data2_list)>0:
        for i in data2_list:
            i = i.replace('/10','')
            i = regex.sub('', str(i))
            if i != '':
                rate_list.append(int(i))
            else:
                rate_list.append(0)
    if len(rate_list)>0:
        result['max_rate'] = max(rate_list) 
    else:
        result['max_rate']=0
        
    if len(dist_list)==len(rate_list):
        result['average_rate_school']=np.array(dist_list)@np.array(rate_list)
    else:
        result['average_rate_school']=0
    
    return pd.Series(result)

In [12]:
def make_stories(row):
    regex_sq = re.compile('\D+')
    try:
        row=float(row)
    except:
        row=regex_sq.sub('', str(row))
        if row != '':
            row = float(row)
        else:
            row = 1
    return row

In [13]:
def make_facts(row, columns):
    fact_dict = {}
    for i in columns:
        fact_dict[i]=float('nan')
    row = eval(row)['atAGlanceFacts']
    for i in row:
        fact_dict[i['factLabel']]=i['factValue']
    return pd.Series(fact_dict)

In [14]:
#в цене за кв. фут избавляемся от всяких лишних знаков, переводим в целое число
def make_price(row):
    row = str(row).replace('$','').replace('/sqft','').replace(' / Sq. Ft.', '').replace(',','')
    try:
        row = int(row)
    except:
        if row in ['None', 'No Info', 'No Data', '', 'Contact manager']:
            row = 0
        else:
            row = 0
    return row

In [15]:
#в размере лота убираем лишние знаки, переводим все в единые единицы измерения
def make_lot(row):
    row1 = row
    row = str(row).replace('$','').replace(' sqft lot','').replace(' Sq. Ft.', '').replace(' sqft', '').replace(',','')
    try:
        row = float(row)
    except:
        if row.lower().find('acre') >-1:
            row = row.lower().replace(' lot', '').replace(' acres', '').replace(' acre', '').replace(' acres', '')
        try:
            row = float(row) * 43560.04
        except:
            if row in ['None', 'No Info', 'No Data', '', 'Contact manager', '--', '—']:
                row = 0
            else:
                print(row, '->', row1)
    return row

## Функция для преобразования строки в нужную нам для предсказания

In [18]:
#на вход у нас строка в виде Series
#на выходе должны быть такие колонки
#Index(['8', 'max_rate', 'PK', 'prop_type', 'address_hash', 'beds_square',
#       'stories_count', 'K', 'Parking', 'Year', 'pool', '6', 'new_id',
#       'beds_count', 'Remodeled', '1', 'state_mean', 'city_hash', '12',
#       'zipcode_int', 'Cooling', 'max_dist', 'street_hash', 'schools_count',
#       '11', 'lot', 'sqft', 'fireplace_y_n', 'sqft_lot', 'average_rate_school',
#       'mean_dist', 'min_dist', 'baths_count', 'state_encoding']
def make_data_4_predict(stroka, states_list,mean_target):
    data = pd.DataFrame(stroka).T
    
    #1
    if data['state'].iloc[0] in mean_target:
        data['state_mean']=mean_target[data['state'].iloc[0]]
    else:
        data['state_mean']=0
    
    #2
    data['rent_sale']=data['status'].apply(lambda x: 1 if str(x).lower().find('rent') >-1 else 0)
    
    #3
    data['prop_type']=data['propertyType'].apply(lambda x: replace_property(x))
    #считаем hash
    hash_space=50
    data['prop_type']=data['prop_type'].apply(lambda x: hash(''.join(str(x).lower().split())) % hash_space)
    
    #4
    # заполняем пропущенные значения на "no_city"
    data['city'].fillna('no_city', inplace=True)
    # приводим все буквы к нижнему регистру, так как есть вероятность, что города могут быть написаны по-разному
    data['city_lower'] = data['city'].apply(lambda x: str(x).lower() if x != ' ' else 'no_city')
    hash_space = 2020
    data['city_hash']=data['city'].apply(lambda x: hash(''.join(x.lower().split())) % hash_space)
    data['street_hash']=data['street'].apply(lambda x: hash(''.join(str(x).lower().split())) % hash_space)
    data['address_hash']=data[['city', 'street']].apply(lambda x: hash(''.join(str(x[0]).lower().split()) + 
                                                                  ''.join(str(x[1]).lower().split())) % hash_space, axis=1)
    
    #5
    data['baths_count']=data['baths'].apply(lambda x: make_baths_count(x))
    
    #6
    data['beds_count']=data['beds'].apply(lambda x: make_beds_count(x))
    data['beds_square']=data['beds'].apply(lambda x: make_beds_square(x))
    
    #7
    #добавляем кучу новых столбцов с характеристиками по школам
    new_columns = ['K','1','2','3','4','5','6','7','8','9','10','11','12', 'PK', 'schools_count', 
              'min_dist', 'max_dist', 'mean_dist', 'max_rate', 'average_rate_school']
    data[new_columns] = pd.DataFrame(data['schools'].apply(lambda x: make_schools(x)), index=data.index)
    
    #8
    data['sqft'].fillna(0, inplace=True)
    #убираем все символы, кроме цифр. Приводим к формату int
    regex_sq = re.compile('\D+')
    data['sqft'] = data['sqft'].apply(lambda x: regex_sq.sub('', str(x)))
    data['sqft'] = data['sqft'].apply(lambda x: int(x) if x != '' else 0)
    
    #9
    #убираем все символы, кроме цифр. Приводим к формату int
    regex = re.compile('\D+')
    data['zipcode_int'] = data['zipcode'].apply(lambda x: int(regex.sub('', str(x))) if regex.sub('', str(x))!='' else 0)
    
    #10
    data['private pool1'] = data['private pool'].apply(lambda x: 1 if str(x).lower()=='yes' else 0)
    data['PrivatePool1'] = data['PrivatePool'].apply(lambda x: 1 if str(x).lower()=='yes' else 0)
    data['pool'] = data[['PrivatePool1','private pool1']].values.max(1)
    
    #11
    data['fireplace_y_n'] = data['fireplace'].apply(lambda x: 0 if (str(x) == 'nan' 
                                                                or str(x).lower() == 'not applicable' 
                                                                or str(x).lower() == 'no') else 1)
    #посчитаем, сколько характеристик указано для дома
    data['fp_count']=data['fireplace'].apply(lambda x: len(str(x).split(',')))
    
    #12
    # заменяем нан на 1 этаж
    data['stories'].fillna(1, inplace=True)
    #убираем все символы, кроме цифр. Приводим к формату int
    regex_sq = re.compile('\D+')
    data['stories_count']=data['stories'].apply(lambda x: make_stories(x))
    
    #13
    states_list = list(states_list)
    data['state_encoding']=data['state'].apply(lambda x: states_list.index(x) if x in states_list else len(states_list))
    
    #14
    fact_columns = ['Cooling',  'Heating', 'Parking', 'Price/sqft', 'Remodeled year', 'Year built', 'lotsize']
    data[fact_columns]=data['homeFacts'].apply(lambda x: make_facts(x,fact_columns))
    #Cooling, Heating, Parking меняем на 1 (есть), 0(нет)
    for i in ['Cooling', 'Heating', 'Parking']:
        data[i].fillna(0, inplace = True)
        data[i] = data[i].apply(lambda x: 0 if x == np.nan or x in ['', 'No','None', 'no', 0, 'No Data'] else 1)
    data['price']=data['Price/sqft'].apply(lambda x: make_price(x))
    #преобразуем год в число
    for i in ['Remodeled year', 'Year built']:
        data[i.split()[0]] = data[i].apply(lambda x: int(x) if str(x) not in ['', 'None','No Data'] else 0)
    data['lot'] = data['lotsize'].apply(lambda x: make_lot(x))
    
    #15
    #соединяем идентификаторы в одну колонку, так удобнее с ними работать
    data['mls-id'].fillna('', inplace=True)
    data['MlsId'].fillna('', inplace=True)
    data['new_id']=data['mls-id'].astype(str)+data['MlsId'].astype(str)
    hash_space = 500000
    data['new_id'] = data['new_id'].apply(lambda x: hash(''.join(x.lower().split())) % hash_space)
    
    data['price_sqft'] = data['price']*data['sqft']
    data['price_lot']=data['price']*data['lot']
    
    data['sqft_lot'] = data['sqft']+data['lot']
    
    columns = ['8', 'max_rate', 'PK', 'prop_type', 'address_hash', 'beds_square',
       'stories_count', 'K', 'Parking', 'Year', 'pool', '6', 'new_id',
       'beds_count', 'Remodeled', '1', 'state_mean', 'city_hash', '12',
       'zipcode_int', 'Cooling', 'max_dist', 'street_hash', 'schools_count',
       '11', 'lot', 'sqft', 'fireplace_y_n', 'sqft_lot', 'average_rate_school',
       'mean_dist', 'min_dist', 'baths_count', 'state_encoding']
    result = data[columns]
    
    return result
    

Проверим, как работает

In [19]:
make_data_4_predict(stroka, states_list,mean_target)

,8,max_rate,PK,prop_type,address_hash,beds_square,stories_count,K,Parking,Year,...,11,lot,sqft,fireplace_y_n,sqft_lot,average_rate_school,mean_dist,min_dist,baths_count,state_encoding
61,1.0,7.0,0.0,33,1992,0,1.0,1.0,0,1983,...,1.0,12632.4116,1446,1,14078.4116,35.8,3.066667,2.2,2.0,4


## Предсказание для данных из файла

In [20]:
data_test = pd.read_csv('data.csv')
y_pred = []

for i in range(0, len(data_test)):
    X_pred = make_data_4_predict(data_test.iloc[i], states_list,mean_target)
    y_pred.append(model.predict(X_pred))


[1154115.94128031] 1,650,000
[391053.39385695] 335,000
[1249397.49211493] 2,650,000
[592632.43542118] $365,000
[447614.5346544] $626,000
[358552.4742348] $375,000
[2194527.44499014] $3,500,000
[340001.9993708] 579,000
[1125837.47010803] $499,007
[386459.10392542] $182,000
[4547745.53257203] $3,749,000
[341943.4314798] 799,000
[449572.25972004] $499,900
[565470.61207978] $559,000
[806696.08330697] $830,000
[870073.39115385] $1,195,000
[260101.96958292] $262,000
[135343.78979912] $204,800
[138853.43253491] $179,000
[71760.56815545] $105,000


# Прототип 2 версия

Предполагаем, что нам передаются не все данные. Поэтому нам надо каким-то образом дополнить недостающее

In [102]:
mean_target['AL']

72000.0

In [161]:
def make_full_data(stroka, mean_target):

    mean_lotsize = 0
    
    #первым делом проверяем, есть ли штат
    if 'state' in stroka.keys():
        if 'homeFacts' in stroka.keys():
            #проверяем, есть ли цена за кв. фут
            s = eval(stroka['homeFacts'])['atAGlanceFacts']
            for k in s:
                if k['factLabel'] == 'Price/sqft':
                    mean_price =k['factValue'] 
        else: #штат есть, но цены нет
            if stroka['state'] in mean_target.index:
                mean_price = mean_target.get([stroka['state']], 0).iloc[0]
            else:
                mean_price = 0
    else: #даже штата нет
        mean_price = mean_target.get('FL', 0)
    
    #некоторое начальное значение словаря
    dict_init = {
        'status': 'for sale', #статус нам неизвестен
        'private pool': 'NaN', #наиболее часто встречающееся
        'propertyType': 'single family home', #наиболее часто встречающееся
        'street':'NaN', #NaN
        'baths':'2.0', #наиболее часто встречающееся
        #наиболее часто встречающееся, но! если знаем штат и город, то лучше среднее по этим параметрам
        #кроме того, если не указана цена за кв. фут, то лучше ее заменить на среднее по штату
        'homeFacts': "{'atAGlanceFacts': [{'factValue': '0', 'factLabel': 'Year built'}, {'factValue': '0', 'factLabel': 'Remodeled year'}, {'factValue': 'NaN', 'factLabel': 'Heating'}, {'factValue': '0', 'factLabel': 'Cooling'}, {'factValue': 'No Data', 'factLabel': 'Parking'}, {'factValue': '"+str(mean_lotsize)+"', 'factLabel': 'lotsize'}, {'factValue': '"+str(mean_price)+"', 'factLabel': 'Price/sqft'}]}",
        'fireplace':'Yes', #наиболее часто встречающееся
        'city':'NaN', #неизвестен
        'schools':"[{'rating': [], 'data': {'Distance': [], 'Grades': []}, 'name': []}]", #по нулям
        'sqft':'2579.4076128911624', #среднее 
        'zipcode':'32137', #nan
        'beds':'3', #наиболее часто встречающееся
        'state':'FL', #наиболее часто встречающееся
        'stories':'1', #наиболее часто встречающееся
        'mls-id':'NaN',
        'PrivatePool':'NaN', #наиболее часто встречающееся
        'MlsId':'NaN',
        'target':'NaN'
        }
    
    for i in stroka.keys():
        dict_init[i] = stroka[i]
    
    return dict_init

In [162]:
stroka2_dict = {
    #'status': 'NaN',
    'private pool': 'NaN',
    'propertyType': 'Single Family',
    #'street':'7353 Boysenberry Ct',
    'baths':'2.0',
    #'homeFacts': "{'atAGlanceFacts': [{'factValue': '1983', 'factLabel': 'Year built'}, {'factValue': '1983', 'factLabel': 'Remodeled year'}, {'factValue': 'Forced air', 'factLabel': 'Heating'}, {'factValue': 'Central', 'factLabel': 'Cooling'}, {'factValue': 'No Data', 'factLabel': 'Parking'}, {'factValue': '0.29 acres', 'factLabel': 'lotsize'}, {'factValue': 'None', 'factLabel': 'Price/sqft'}]}",
    'fireplace':'Yes',
    #'city':'Jacksonville',
    #'schools':"[{'rating': ['7/10', '2/10', '4/10'], 'data': {'Distance': ['2.2 mi', '3.8 mi', '3.2 mi'], 'Grades': ['K-5', '6-8', '9-12']}, 'name': ['Chimney Lakes Elementary School', 'Jefferson Davis Middle School', 'Westside High School']}]",
    #'sqft':'1,446 sqft',
    #'zipcode':'32244',
    #'beds':'3',
    #'state':'FL',
    'stories':'1',
    #'mls-id':'NaN',
    'PrivatePool':'NaN',
    'MlsId':'NaN',
    'target':'NaN'
    }

In [163]:
model.predict(make_data_4_predict(pd.Series(make_full_data(stroka2_dict)), states_list,mean_target))

array([5211086.25623629])